### Model to predict if a customer exited from a bank or not based on different parameters like credit score

In [1]:
# Importing the dataset
dataset = read.csv('Churn_Modelling.csv')
dataset = dataset[4:14]

In [2]:
head(dataset)

CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,France,Female,42,2,0.00,1,1,1,101348.88,1
608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
502,France,Female,42,8,159660.80,3,1,0,113931.57,1
699,France,Female,39,1,0.00,2,0,0,93826.63,0
850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1


In [3]:
# Encoding the categorical variables as factors
dataset$Geography = as.numeric(factor(dataset$Geography,
                                      levels = c('France', 'Spain', 'Germany'),
                                      labels = c(1, 2, 3)))
dataset$Gender = as.numeric(factor(dataset$Gender,
                                   levels = c('Female', 'Male'),
                                   labels = c(1, 2)))

In [4]:
# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(dataset$Exited, SplitRatio = 0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

Warning message:
"package 'caTools' was built under R version 3.4.3"

In [5]:
# Fitting XGBoost to the Training set
# install.packages('xgboost')
library(xgboost)
classifier = xgboost(data = as.matrix(training_set[-11]), label = training_set$Exited, nrounds = 10)

Warning message:
"package 'xgboost' was built under R version 3.4.3"

[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.298272 


In [6]:
# Predicting the Test set results
y_pred = predict(classifier, newdata = as.matrix(test_set[-11]))
y_pred = (y_pred >= 0.5)

In [7]:
# Making the Confusion Matrix
cm = table(test_set[, 11], y_pred)

In [8]:
# Applying k-Fold Cross Validation
# install.packages('caret')
library(caret)
folds = createFolds(training_set$Exited, k = 10)
cv = lapply(folds, function(x) {
  training_fold = training_set[-x, ]
  test_fold = training_set[x, ]
  classifier = xgboost(data = as.matrix(training_set[-11]), label = training_set$Exited, nrounds = 10)
  y_pred = predict(classifier, newdata = as.matrix(test_fold[-11]))
  y_pred = (y_pred >= 0.5)
  cm = table(test_fold[, 11], y_pred)
  accuracy = (cm[1,1] + cm[2,2]) / (cm[1,1] + cm[2,2] + cm[1,2] + cm[2,1])
  return(accuracy)
})
accuracy = mean(as.numeric(cv))

Warning message:
"package 'caret' was built under R version 3.4.3"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.4.3"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.4.3"

[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.298272 
[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.298272 
[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.298272 
[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.2982

In [9]:
cm

   y_pred
    FALSE TRUE
  0  1541   52
  1   211  196

In [10]:
accuracy

[1] 0.88075